# Merlion

In [ ]:
import numpy as np
import pandas as pd
import json
import lxml.html

import datetime

## getCatalog

In [ ]:
file = open('/home/abezpalov/data/merlion/categories.json', 'r')
response = json.load(file)

In [ ]:
def get_order(x):
    return int(x[-2:])

# Преобразуем формат данных
data_list = list()
for item in response:
    row_ = dict()
    for key in item:
        row_[key] = item[key]
    data_list.append(row_)
df = pd.DataFrame(data_list)

# Чистим данные
df = df.rename(columns={'ID': 'key',
                        'ID_PARENT': 'parent_key',
                        'Description': 'name'})

df['order'] = df['key'].apply(get_order)

In [ ]:
df

## getItems

In [ ]:
file = open('/home/abezpalov/data/merlion/products_32.json', 'r')
response = json.load(file)

In [ ]:
data_list = list()
for item in response:
    row_ = dict()
    for key in item:
        row_[key] = item[key]
    data_list.append(row_)

df = pd.DataFrame(data_list)

### Смотрим данные

In [ ]:
list(df)

In [ ]:
display(df['No'].describe())
display(df['No'].unique())

In [ ]:
display(df['Min_Packaged'].describe())
print(set(df['Min_Packaged'].unique()))

In [ ]:
display(df['Sales_Limit_Type'].describe())
print(set(df['Sales_Limit_Type'].unique()))

In [ ]:
df[df['Sales_Limit_Type'] != 'Только Упаковками']

### Чистим данные

In [ ]:
def get_category_key(row):

    if row['GroupCode3']:
        return row['GroupCode3']
    elif row['GroupCode2']:
        return row['GroupCode2']
    return row['GroupCode1']

def get_min_of_quantity(row):
    if not row['Min_Packaged']:
        return 1
    elif row['Sales_Limit_Type'] in ('Кратно', 'Только Упаковками', 'Не Меньше'):
        return row['Min_Packaged']
    return 1

def get_step_of_quantity(row):
    if not row['Min_Packaged']:
        return 1
    elif row['Sales_Limit_Type'] in ('Кратно', 'Только Упаковками'):
        return row['Min_Packaged']
    return 1

def get_eol(x):
    return True if x == 1 else False

def get_vat(x):
    return None if x is None else int(x)

def get_length(row):
    return max(row['Length'], row['Width']) if row['Length'] and row['Width'] else None
    
def get_width(row):
    return min(row['Length'], row['Width']) if row['Length'] and row['Width'] else None

def get_promo(row):
    return True if row['promo_name'] or row['promo_url'] else False

def get_metadata(row):
    metadata = dict(row[['IsBundle', 'IsNew', 'Last_time_modified']])
    return json.dumps(metadata)

df.rename(columns = {'No': 'key',
                     'Name': 'name',
                     'Brand': 'vendor_key',
                     'Vendor_part': 'part_number',
                     'Warranty': 'warranty',
                     'Weight': 'weight',
                     'Volume': 'volume',
                     'Height': 'height',
                     'ActionDesc': 'promo_name',
                     'ActionWWW': 'promo_url'}, inplace=True)
df['category_key'] = df.apply(get_category_key, axis=1)
df['min_of_quantity'] = df.apply(get_min_of_quantity, axis=1)
df['step_of_quantity'] = df.apply(get_step_of_quantity, axis=1)
df['eol'] = df['EOL'].apply(get_eol)
df['vat'] = df['VAT'].apply(get_vat)
df['length'] = df.apply(get_length, axis=1)
df['width'] = df.apply(get_width, axis=1)
df['promo'] = df.apply(get_promo, axis=1)
df['metadata'] = df.apply(get_metadata, axis=1)

df = df.drop(columns=['Size', 'GroupName1', 'GroupName2', 'GroupName3', 'GroupCode1', 'GroupCode2', 'GroupCode3', 'Sales_Limit_Type',
                      'Min_Packaged', 'EOL', 'VAT', 'Length', 'Width', 'IsBundle', 'IsNew', 'Last_time_modified'])

In [ ]:
df

## getItemsAvail

In [ ]:
file = open('/home/abezpalov/data/merlion/parties_43.json', 'r')
response = json.load(file)

In [ ]:
data_list = list()
for item in response:

    row_ = dict()
    for key in item:
        row_[key] = item[key]
    data_list.append(row_)

df = pd.DataFrame(data_list)

### Смотрим данные

In [ ]:
df

In [ ]:
list(df)

In [ ]:
display(df['RRP_Date'].describe())
display(df['RRP_Date'].unique())

### Чистим данные

In [ ]:
def get_price_in(row):
    if row['PriceClient']:
        return row['PriceClient']
    elif row['PriceClientRUB']:
        return row['PriceClientRUB']
    return None

def get_currency_key_in(row):
    if row['PriceClient']:
        return 'USD'
    elif row['PriceClientRUB']:
        return 'RUB'
    return None

def get_price_out(row):
    return float(row['RRP']) if row['RRP'] and row['RRP'] > row['PriceClientRUB'] else None

def get_currency_key_out(row):
    return 'RUB' if row['RRP'] and row['RRP'] > row['PriceClientRUB'] else None

def get_can_reserve(x):
    return True if x == 0.0 else False

def get_can_order(x):
    return True if  x == 0.0 else False

def get_quantity(x):
    return int(float(x))

def get_expected_date(x):
    return None if x is None else datetime.datetime.strptime(x, "%Y-%m-%d").date()

def get_metadata(row):
    metadata = dict(row[['ReserveCost']])
    return json.dumps(metadata)

df = df.rename(columns = {'No': 'products_key'})
df['price_in'] = df.apply(get_price_in, axis=1)
df['currency_key_in'] = df.apply(get_currency_key_in, axis=1)
df['price_out'] = df.apply(get_price_out, axis=1)
df['currency_key_out'] = df.apply(get_currency_key_out, axis=1)
df['can_reserve'] = df['Online_Reserve'].apply(get_can_reserve)
df['can_order'] = df['Online_Reserve'].apply(get_can_order)
df['metadata'] = df.apply(get_metadata, axis=1)

dfs = list()

# Проверяем наполнение регионального склада
df_ = df[df['AvailableClient_RG'] > 0].copy()
df_['quantity'] = df_['AvailableClient_RG'].apply(get_quantity)
df_ = df_.assign(location_key='Региональный склад')
df_ = df_.assign(expected_date=None)
dfs.append(df_)

# Проверяем наполнение центрального склада
df_ = df[df['AvailableClient_MSK'] > 0].copy()
df_['quantity'] = df_['AvailableClient_MSK'].apply(get_quantity)
df_ = df_.assign(location_key='Московский склад')
df_ = df_.assign(expected_date=None)
dfs.append(df_)

df_ = df[df['AvailableExpected'] > 0].copy()
df_['quantity'] = df_['AvailableExpected'].apply(get_quantity)
df_ = df_.assign(location_key='Транзит')
df_['expected_date'] = df_['DateExpectedNext'].apply(get_expected_date)
dfs.append(df_)

df_ = df[df['AvailableExpectedNext'] > 0].copy()
df_['quantity'] = df_['AvailableExpectedNext'].apply(get_quantity)
df_ = df_.assign(location_key='Транзит')
df_['expected_date'] = df_['DateExpectedNext'].apply(get_expected_date)
dfs.append(df_)

df = pd.concat(dfs)


df = df.drop(columns=['PriceClient', 'PriceClient_RG', 'PriceClient_MSK', 'PriceClientRUB', 'PriceClientRUB_RG', 'PriceClientRUB_MSK',
                     'RRP', 'RRP_Date', 'Online_Reserve', 'AvailableClient', 'AvailableClient_RG', 'AvailableClient_MSK',
                      'AvailableExpected', 'AvailableExpectedNext', 'DateExpectedNext', 'ReserveCost'])

In [ ]:
df

## getItemsBarcodes

In [ ]:
file = open('/home/abezpalov/data/merlion/barcodes_725.json', 'r')
response = json.load(file)

data_list = list()
for item in response:

    row = dict()
    for key in item:
        row[key] = item[key]
    data_list.append(row)

df = pd.DataFrame(data_list)

### Смотрим данные

In [ ]:
df

In [ ]:
list(df)

### Чистим данные

In [ ]:
def get_packaging(x):
    result = {'ШТ': 'штучный товар',
              'BOX': 'стандартная упаковка (BOX)',
              'BOXX': 'упаковка (BOXX)',
              'IBOX': 'промежуточная упаковка (IBOX)',
              'BBOX': 'альтернативная упаковка (BBOX)',
              'BBOXX': 'альтернативная упаковка (BBOXX)'}
    try:
        return None if x is None else result[x]
    except KeyError:
        return x

df = df.rename(columns = {'ItemNo': 'product_key',
                          'Barcode': 'value'})

df['checked'] = df['Checked'].apply(bool)
df['packaging'] = df['UnitOfMeasure'].apply(get_packaging)
df = df.assign(form="EAN 128")
df = df.drop(columns=['UnitOfMeasure', 'Checked'])

In [ ]:
df

## getItemsPropertiesSections

In [ ]:
file = open('/home/abezpalov/data/merlion/parameter_groups.json', 'r')
response = json.load(file)

data_list = list()
for item in response:

    row = dict()
    for key in item:
        row[key] = item[key]
    data_list.append(row)

df = pd.DataFrame(data_list)

In [ ]:
df = df.rename(columns={'id': 'key'})

In [ ]:
file = open('/home/abezpalov/data/merlion/parameters_32.json', 'r')
response = json.load(file)

data_list = list()
for item in response:

    row = dict()
    for key in item:
        row[key] = item[key]
    data_list.append(row)

df = pd.DataFrame(data_list)

In [ ]:
df

## getItemsProperties

In [ ]:
file = open('/home/abezpalov/data/merlion/parameters_724.json', 'r')
response = json.load(file)

data_list = list()
for item in response:

    row = dict()
    for key in item:
        row[key] = item[key]
    data_list.append(row)

df = pd.DataFrame(data_list)

In [ ]:
df = df.rename(columns={'No': 'product_key',
                        'PropertyID': 'parameter_key',
                        'PropertyName': 'name',
                        'Sorting': 'order',
                        'Value': 'value',
                        'Measure_Name': 'unit_key',
                        'Section_Id': 'group_key'})

df = df.drop(columns=['Last_time_modified'])

dfs = dict()

dfs['parameters'] = df[['group_key', 'parameter_key', 'name', 'order']]
dfs['parameters'] = dfs['parameters'].rename(columns={'parameter_key': 'key'})

dfs['values'] = df[['product_key', 'group_key', 'parameter_key', 'value', 'unit_key']]

In [ ]:
display(dfs['parameters'])
display(dfs['values'])

## getItemsImages

In [ ]:
file = open('/home/abezpalov/data/merlion/images_43.json', 'r')
response = json.load(file)

data_list = list()
for item in response:

    row = dict()
    for key in item:
        row[key] = item[key]
    data_list.append(row)

df = pd.DataFrame(data_list)

In [ ]:
def get_source_url(x):
    return f'http://img.merlion.ru/items/{x}'

df['source_url'] = df['FileName'].apply(get_source_url)
df = df.rename(columns={'No': 'product_key',
                        'Size': 'size',
                        'Width': 'width',
                        'Height': 'height'})
df = df.drop(columns=['ViewType', 'SizeType', 'FileName', 'Created'])

In [ ]:
df